# Model 5: Activation Function

This notebook contains the construction and training of the model iterations and various experiments. The notebook is split up into four sections: training mode selection (where the model will run), set-up, model constrution, and training. 

Evaluation will take place in the notebook *model_optimization_and_evaluation.ipynb* found in the *notebooks* folder.

In [1]:
# importing necessary packages and libraries
import numpy as np
import pandas as pd
from keras import layers
from keras import models
from keras import optimizers
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.generic_utils import get_custom_objects
from keras.callbacks import ModelCheckpoint
import pickle

Using TensorFlow backend.


## Step 1: Training Mode Selection

In the cell below, specify the training mode for the model. This will determine the location from which the source data is drawn, and to which the trained models (and training histories) are saved. 

- **training_mode = 'floydhub'** (runs on Floydhub)
- **training_mode = 'local'** (runs on local disk and processor)

In [2]:
# select training mode
training_mode = 'floydhub'

## Step 2: Set-up

In [3]:
# directory base paths
data_path_local = '../../data/0002_array_data/train_data/'
model_path_local = '../../notebooks/model_construction/saved_models/'
data_path_floydhub = '/floyd/input/capstone_mushrooms/'
model_path_floydhub = '/floyd/home/'

# setting directory paths based on training mode selection
if training_mode == 'floydhub':
    data_path = data_path_floydhub
    model_path = model_path_floydhub
elif training_mode == 'local':
    data_path = data_path_local
    model_path = model_path_local
else:
    raise Exception('Please choose valid training mode: "floydhub" or "local".')

In [4]:
# loading the training and validation data subsets
X_train = np.load(f'{data_path}X_train_data.npy')
y_train = np.load(f'{data_path}y_train_data.npy')
X_val = np.load(f'{data_path}X_val_data.npy')
y_val = np.load(f'{data_path}y_val_data.npy')

In [5]:
# setting training parameters
batch_size = 8
n_classes = 20
n_epochs = 15
img_shape = X_train.shape[1:]
model_names = []
model_list = []
model_hists = []

## Step 3: Model Construction

### Trial 1 - ELU

In [6]:
# defining the model architecture
m5_t1 = models.Sequential()

# convolution/max pool stacks
m5_t1.add(layers.Conv2D(32,(3,3), input_shape=img_shape, padding='same'))
m5_t1.add(layers.ELU(alpha=0.1))
m5_t1.add(layers.MaxPooling2D((2,2)))

m5_t1.add(layers.Conv2D(64,(3,3), padding='same'))
m5_t1.add(layers.ELU(alpha=0.1))
m5_t1.add(layers.MaxPooling2D((2,2)))

m5_t1.add(layers.Conv2D(128,(3,3), padding='same'))
m5_t1.add(layers.ELU(alpha=0.1))
m5_t1.add(layers.MaxPooling2D((2,2)))

m5_t1.add(layers.Conv2D(256,(3,3), padding='same'))
m5_t1.add(layers.ELU(alpha=0.1))
m5_t1.add(layers.MaxPooling2D((2,2)))

# fully connected layers
m5_t1.add(layers.Flatten())
m5_t1.add(layers.Dense(512))
m5_t1.add(layers.ELU(alpha=0.1))
m5_t1.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
m5_t1.summary()
model_names.append('m5_t1')
model_list.append(m5_t1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
elu_1 (ELU)                  (None, 200, 200, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
elu_2 (ELU)                  (None, 100, 100, 64)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 128)      

### Trial 2 - PReLU

In [7]:
# defining the model architecture
m5_t2 = models.Sequential()

# convolution/max pool stacks
m5_t2.add(layers.Conv2D(32,(3,3), input_shape=img_shape, padding='same'))
m5_t2.add(layers.PReLU())
m5_t2.add(layers.MaxPooling2D((2,2)))

m5_t2.add(layers.Conv2D(64,(3,3), padding='same'))
m5_t2.add(layers.PReLU())
m5_t2.add(layers.MaxPooling2D((2,2)))

m5_t2.add(layers.Conv2D(128,(3,3), padding='same'))
m5_t2.add(layers.PReLU())
m5_t2.add(layers.MaxPooling2D((2,2)))

m5_t2.add(layers.Conv2D(256,(3,3), padding='same'))
m5_t2.add(layers.PReLU())
m5_t2.add(layers.MaxPooling2D((2,2)))

# fully connected layers
m5_t2.add(layers.Flatten())
m5_t2.add(layers.Dense(512))
m5_t2.add(layers.PReLU())
m5_t2.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
m5_t2.summary()
model_names.append('m5_t2')
model_list.append(m5_t2)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
p_re_lu_1 (PReLU)            (None, 200, 200, 32)      1280000   
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
p_re_lu_2 (PReLU)            (None, 100, 100, 64)      640000    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 50, 50, 128)      

### Trial 3 - Leaky ReLU

In [8]:
# defining the model architecture
m5_t3 = models.Sequential()

# convolution/max pool stacks
m5_t3.add(layers.Conv2D(32,(3,3), input_shape=img_shape, padding='same'))
m5_t3.add(layers.LeakyReLU(alpha=0.1))
m5_t3.add(layers.MaxPooling2D((2,2)))

m5_t3.add(layers.Conv2D(64,(3,3), padding='same'))
m5_t3.add(layers.LeakyReLU(alpha=0.1))
m5_t3.add(layers.MaxPooling2D((2,2)))

m5_t3.add(layers.Conv2D(128,(3,3), padding='same'))
m5_t3.add(layers.LeakyReLU(alpha=0.1))
m5_t3.add(layers.MaxPooling2D((2,2)))

m5_t3.add(layers.Conv2D(256,(3,3), padding='same'))
m5_t3.add(layers.LeakyReLU(alpha=0.1))
m5_t3.add(layers.MaxPooling2D((2,2)))

# fully connected layers
m5_t3.add(layers.Flatten())
m5_t3.add(layers.Dense(512))
m5_t3.add(layers.LeakyReLU(alpha=0.1))
m5_t3.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
m5_t3.summary()
model_names.append('m5_t3')
model_list.append(m5_t3)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 200, 200, 32)      0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 100, 100, 64)      18496     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 100, 100, 64)      0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 50, 50, 128)      

### Trial 4 - Swish

In [9]:
# importing additional libraries
from keras import backend as K

# updating activations class with swish function
class Swish(layers.Activation):   
    def __init__(self, activation, **kwargs):
        super(Swish, self).__init__(activation, **kwargs)
        self.__name__ = 'swish'

# defining swish function        
def swish(x):
    return (K.sigmoid(x)*x)

# updating custom objects with swish function
get_custom_objects().update({'swish': Swish(swish)})

In [10]:
# defining the model architecture
m5_t4 = models.Sequential()

# convolution/max pool stacks
m5_t4.add(layers.Conv2D(32,(3,3), activation='swish', input_shape=img_shape, padding='same'))
m5_t4.add(layers.MaxPooling2D((2,2)))

m5_t4.add(layers.Conv2D(64,(3,3), activation='swish', padding='same'))
m5_t4.add(layers.MaxPooling2D((2,2)))

m5_t4.add(layers.Conv2D(128,(3,3), activation='swish', padding='same'))
m5_t4.add(layers.MaxPooling2D((2,2)))

m5_t4.add(layers.Conv2D(256,(3,3), activation='swish', padding='same'))
m5_t4.add(layers.MaxPooling2D((2,2)))

# fully connected layers
m5_t4.add(layers.Flatten())
m5_t4.add(layers.Dense(512, activation='swish'))
m5_t4.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
m5_t4.summary()
model_names.append('m5_t4')
model_list.append(m5_t4)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 200, 200, 32)      896       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 100, 100, 64)      18496     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 50, 50, 128)       73856     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 25, 25, 128)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 25, 25, 256)      

## Step 4: Training

In [11]:
# setting up standardization and augmentation parameters
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.3,
                                   height_shift_range=0.3,
                                   shear_range=0.3,
                                   zoom_range=0.3,
                                   fill_mode='nearest',
                                   horizontal_flip=True,
                                   vertical_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
%%time
# data standardization and augmentation
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size)

CPU times: user 416 ms, sys: 1.3 s, total: 1.72 s
Wall time: 1.72 s


In [13]:
# compiling loss functions
m5_t1.compile(loss='categorical_crossentropy',optimizer='adamax',metrics=['acc'])
m5_t2.compile(loss='categorical_crossentropy',optimizer='adamax',metrics=['acc'])
m5_t3.compile(loss='categorical_crossentropy',optimizer='adamax',metrics=['acc'])
m5_t4.compile(loss='categorical_crossentropy',optimizer='adamax',metrics=['acc'])

In [14]:
%%time
# setting up model saving checkpoints
m5_t1_cp = ModelCheckpoint(filepath=f'{model_path}m5_t1.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
m5_t1_history = m5_t1.fit(train_generator,
                                steps_per_epoch=len(X_train)//batch_size,
                                epochs=n_epochs,
                                callbacks=[m5_t1_cp],
                                validation_data=val_generator,
                                validation_steps=len(X_val)//batch_size)

# adding training history to list
model_hists.append(m5_t1_history)

Epoch 1/15
1013/1013 [==============================] - 92s 91ms/step - loss: 2.6411 - acc: 0.1696 - val_loss: 2.2830 - val_acc: 0.3131

Epoch 00001: val_acc improved from -inf to 0.31312, saving model to /floyd/home/m5_t1.h5
Epoch 2/15
1013/1013 [==============================] - 90s 89ms/step - loss: 2.2936 - acc: 0.2679 - val_loss: 2.7552 - val_acc: 0.3429

Epoch 00002: val_acc improved from 0.31312 to 0.34289, saving model to /floyd/home/m5_t1.h5
Epoch 3/15
1013/1013 [==============================] - 90s 89ms/step - loss: 2.1731 - acc: 0.3004 - val_loss: 2.5141 - val_acc: 0.3504

Epoch 00003: val_acc improved from 0.34289 to 0.35037, saving model to /floyd/home/m5_t1.h5
Epoch 4/15
1013/1013 [==============================] - 90s 89ms/step - loss: 2.0734 - acc: 0.3346 - val_loss: 1.3026 - val_acc: 0.4514

Epoch 00004: val_acc improved from 0.35037 to 0.45137, saving model to /floyd/home/m5_t1.h5
Epoch 5/15
1013/1013 [==============================] - 90s 89ms/step - loss: 1.9871 - 

In [15]:
%%time
# setting up model saving checkpoints
m5_t2_cp = ModelCheckpoint(filepath=f'{model_path}m5_t2.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
m5_t2_history = m5_t2.fit(train_generator,
                                steps_per_epoch=len(X_train)//batch_size,
                                epochs=n_epochs,
                                callbacks=[m5_t2_cp],
                                validation_data=val_generator,
                                validation_steps=len(X_val)//batch_size)

# adding training history to list
model_hists.append(m5_t2_history)

Epoch 1/15
1013/1013 [==============================] - 79s 78ms/step - loss: 2.7400 - acc: 0.1305 - val_loss: 2.2188 - val_acc: 0.2030

Epoch 00001: val_acc improved from -inf to 0.20297, saving model to /floyd/home/m5_t2.h5
Epoch 2/15
1013/1013 [==============================] - 79s 78ms/step - loss: 2.4536 - acc: 0.1983 - val_loss: 2.0447 - val_acc: 0.2930

Epoch 00002: val_acc improved from 0.20297 to 0.29302, saving model to /floyd/home/m5_t2.h5
Epoch 3/15
1013/1013 [==============================] - 79s 78ms/step - loss: 2.2821 - acc: 0.2599 - val_loss: 2.1920 - val_acc: 0.3416

Epoch 00003: val_acc improved from 0.29302 to 0.34165, saving model to /floyd/home/m5_t2.h5
Epoch 4/15
1013/1013 [==============================] - 79s 78ms/step - loss: 2.1527 - acc: 0.3025 - val_loss: 2.0418 - val_acc: 0.4052

Epoch 00004: val_acc improved from 0.34165 to 0.40524, saving model to /floyd/home/m5_t2.h5
Epoch 5/15
1013/1013 [==============================] - 79s 78ms/step - loss: 2.0537 - 

In [16]:
%%time
# setting up model saving checkpoints
m5_t3_cp = ModelCheckpoint(filepath=f'{model_path}m5_t3.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
m5_t3_history = m5_t3.fit(train_generator,
                                steps_per_epoch=len(X_train)//batch_size,
                                epochs=n_epochs,
                                callbacks=[m5_t3_cp],
                                validation_data=val_generator,
                                validation_steps=len(X_val)//batch_size)

# adding training history to list
model_hists.append(m5_t3_history)

Epoch 1/15
1013/1013 [==============================] - 78s 77ms/step - loss: 2.7005 - acc: 0.1520 - val_loss: 1.5096 - val_acc: 0.2921

Epoch 00001: val_acc improved from -inf to 0.29208, saving model to /floyd/home/m5_t3.h5
Epoch 2/15
1013/1013 [==============================] - 78s 77ms/step - loss: 2.3114 - acc: 0.2533 - val_loss: 1.8426 - val_acc: 0.3716

Epoch 00002: val_acc improved from 0.29208 to 0.37157, saving model to /floyd/home/m5_t3.h5
Epoch 3/15
1013/1013 [==============================] - 78s 77ms/step - loss: 2.1803 - acc: 0.2893 - val_loss: 1.9939 - val_acc: 0.4065

Epoch 00003: val_acc improved from 0.37157 to 0.40648, saving model to /floyd/home/m5_t3.h5
Epoch 4/15
1013/1013 [==============================] - 78s 77ms/step - loss: 2.0828 - acc: 0.3290 - val_loss: 1.6030 - val_acc: 0.4401

Epoch 00004: val_acc improved from 0.40648 to 0.44015, saving model to /floyd/home/m5_t3.h5
Epoch 5/15
1013/1013 [==============================] - 78s 77ms/step - loss: 1.9941 - 

In [17]:
%%time
# setting up model saving checkpoints
m5_t4_cp = ModelCheckpoint(filepath=f'{model_path}m5_t4.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
m5_t4_history = m5_t4.fit(train_generator,
                                steps_per_epoch=len(X_train)//batch_size,
                                epochs=n_epochs,
                                callbacks=[m5_t4_cp],
                                validation_data=val_generator,
                                validation_steps=len(X_val)//batch_size)

# adding training history to list
model_hists.append(m5_t4_history)

Epoch 1/15
1013/1013 [==============================] - 78s 77ms/step - loss: 2.6822 - acc: 0.1537 - val_loss: 2.0849 - val_acc: 0.2921

Epoch 00001: val_acc improved from -inf to 0.29208, saving model to /floyd/home/m5_t4.h5
Epoch 2/15
1013/1013 [==============================] - 78s 77ms/step - loss: 2.4595 - acc: 0.2110 - val_loss: 1.0718 - val_acc: 0.3441

Epoch 00002: val_acc improved from 0.29208 to 0.34414, saving model to /floyd/home/m5_t4.h5
Epoch 3/15
1013/1013 [==============================] - 79s 78ms/step - loss: 2.2976 - acc: 0.2617 - val_loss: 2.2324 - val_acc: 0.3504

Epoch 00003: val_acc improved from 0.34414 to 0.35037, saving model to /floyd/home/m5_t4.h5
Epoch 4/15
1013/1013 [==============================] - 78s 77ms/step - loss: 2.2122 - acc: 0.2912 - val_loss: 2.4057 - val_acc: 0.4077

Epoch 00004: val_acc improved from 0.35037 to 0.40773, saving model to /floyd/home/m5_t4.h5
Epoch 5/15
1013/1013 [==============================] - 78s 77ms/step - loss: 2.1311 - 

In [18]:
# creating dictionary for model names, models, and histories from respective lists
models_dict = {i:[j,k] for i,j,k in zip(model_names,model_list,model_hists)}

In [19]:
# evaluating models on validation set
for key, value in models_dict.items():
    model = models.load_model(f'{model_path}{key}.h5')
    (val_loss, val_accuracy) = model.evaluate(val_generator,verbose=1)
    print(f'{key} Val Accuracy: {round((val_accuracy*100),2)}%')
    print(f'{key} Val Loss: {round(val_loss,4)}')
    print('---')

102/102 [==============================] - 2s 20ms/step
m5_t1 Val Accuracy: 54.57%
m5_t1 Val Loss: 0.055
---
102/102 [==============================] - 2s 23ms/step
m5_t2 Val Accuracy: 54.94%
m5_t2 Val Loss: 1.1891
---
102/102 [==============================] - 2s 18ms/step
m5_t3 Val Accuracy: 59.75%
m5_t3 Val Loss: 0.2868
---
102/102 [==============================] - 2s 17ms/step
m5_t4 Val Accuracy: 51.6%
m5_t4 Val Loss: 0.4262
---


In [20]:
# saving training histories
for key, value in models_dict.items():
    with open(f'{model_path}{key}_history', 'wb') as file_pi:
        pickle.dump(value[1].history, file_pi)
    print(f'{key}_history saved in {model_path}')

m5_t1_history saved in /floyd/home/
m5_t2_history saved in /floyd/home/
m5_t3_history saved in /floyd/home/
m5_t4_history saved in /floyd/home/
